In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as torchvision
from pytorch_lightning.callbacks import EarlyStopping
from torch.nn.functional import cross_entropy
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from pytorch_lightning import loggers as pl_loggers
from torchmetrics import Accuracy
from pytorch_lightning import LightningModule, Trainer

In [2]:
# unzip files
# !unzip "/home/jovyan/work/Assignment_3/files/Test-20241113T152628Z-001.zip" -d "./files"
# !unzip "/home/jovyan/work/Assignment_3/files/Training-20241113T155003Z-001.zip" -d "./files"

In [3]:
# Exercise 4: Data Loading
class DataLoading:
    def __init__(self):
        self.filePath = "./files/"
        self.batch_size = 16

    def load_data(self):

        train_set = torchvision.datasets.ImageFolder(root=self.filePath + "Training",
                                                     transform=torchvision.transforms.Compose(
                                                         [torchvision.transforms.ToTensor(),
                                                          torchvision.transforms.Resize((100, 100))]))

        test_set = torchvision.datasets.ImageFolder(root=self.filePath + "Test",
                                                    transform=torchvision.transforms.Compose(
                                                        [torchvision.transforms.ToTensor(),
                                                         torchvision.transforms.Resize((100, 100))]))
        trainlength = round(0.8 * len(train_set))
        vallength = round(0.2 * len(train_set))
        train_set, val_set = random_split(train_set, [trainlength, vallength])

        train_loader = DataLoader(train_set, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=self.batch_size, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=self.batch_size, shuffle=False)

        return train_loader, val_loader, test_loader

In [10]:
class PLANTSCNN(pl.LightningModule):
# Exercise 1: Convolutional Neural Network Architecture Definition
    def __init__(self):
        super(PLANTSCNN, self).__init__()
        # Layer block 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(5, 5), stride=1, padding="same"),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )

        # Layer block 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=1, padding="same"),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )

        # Layer block 3
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )

        # Layer block 4
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )

        # Flatten layer
        self.flatten = nn.Flatten()
        '''
        For the answer, of why the fully connected layer needs an Input of 128*4*4 comes here the calculation of the convolutional layer output sizes:
        
        Output shape layer1:
        Where:
            HW: Input height and width
            K: Kernel size
            S: Stride
            P: Padding
            MP: MaxPool

            For "same" padding:
            P = K/2
            
           W & H = ((HW - K + 2*P/S)+1)/MP
           
        W & H = ((100-5+2*2/1)+1)/2 = 50
        (16, 50, 50)
        
        Output shape layer2:
        W & H = ((50-3+2*1/1)+1)/2 = 25
        (32, 25, 25)
        
        Output shape layer3:
        W & H = ((25-3+2*0/1)+1)/2 = 11.5 ~ 11
        (64, 11, 11)
        
        Output shape layer4:
        W & H = ((11-3+2*0/1)+1)/2 = 4.5 ~ 4
        (128, 4, 4)
        
        Now the output size of the last conv layer is (128, 4, 4). Those will be flattend to a 1d Vecoter for the input of the
        Fully connected layer 128*4*4 = 2048. So the Input size for the fully connected layer is 2048.
        '''

        # Fully connected layer block
        self.fc = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),  # Adjust if input size differs
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(128, 10)
        )
        
        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.val_acc = Accuracy(task="multiclass", num_classes=10)
        self.test_acc = Accuracy(task="multiclass", num_classes=10)
        self.train_acc_history = []
        self.test_step_outputs = []

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x
    
# Exercise 2: Optimizer
    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=0.01)
        lr_scheduler = StepLR(optimizer=optimizer, step_size=1)
        return [optimizer], [lr_scheduler]
        
# Exercise 3: Training, Validation and Test Step
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = cross_entropy(y_hat, y)
        
        # Update train accuracy
        self.train_acc(y_hat, y)
        
        # Log training loss and accuracy
        self.log("train_loss", loss, on_step=True)
        self.log("train_acc", self.train_acc, on_epoch=True, prog_bar=True)
        
        self.train_acc(y_hat, y)
        return loss
    
    def on_train_epoch_end(self):
        # Compute and log the train accuracy for the epoch
        train_acc_epoch = self.train_acc.compute()
        # Store the accuracy in the history list
        self.train_acc_history.append(train_acc_epoch.item())
        print(f"Epoch {self.current_epoch} - Train Accuracy: {train_acc_epoch:.4f}")
        # Reset metric for the next epoch
        self.train_acc.reset()
        
    def on_train_end(self):
        # Log the max train accuracy achieved
        max_train_acc = max(self.train_acc_history)
        print(f"Max Train Accuracy Achieved: {max_train_acc:.4f}")

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        val_loss = cross_entropy(y_hat, y)
        self.val_acc(y_hat, y)
        self.log("val_loss", val_loss, on_step=False, on_epoch=True)
        self.log("val_acc", self.val_acc, on_epoch=True)
        
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        test_loss = cross_entropy(y_hat, y)
        self.log("test_loss", test_loss, on_step=False, on_epoch=True)
        self.log("test_acc", self.test_acc, on_step=False, on_epoch=True)
        self.test_step_outputs.append({"test_loss": test_loss, "test_acc": self.test_acc(y_hat, y)})

    def on_test_epoch_end(self):
        avg_loss = torch.stack([x["test_loss"] for x in self.test_step_outputs]).mean()
        avg_acc = torch.stack([x["test_acc"] for x in self.test_step_outputs]).mean()
        self.log("avg_test_loss", avg_loss)
        self.log("avg_test_acc", avg_acc)
        self.test_step_outputs.clear()  # Clear the outputs after logging


if __name__ == "__main__":
    # Main function of script
    num_epochs = 200
    
    data_load = DataLoading()
    train_loader, val_loader, test_loader = data_load.load_data()
    
# Exercise 5: Training and Evaluation
    model = PLANTSCNN()

    tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs/")

    trainer = Trainer(log_every_n_steps=10, max_epochs=num_epochs, logger=tb_logger,
                         callbacks=EarlyStopping(monitor="val_loss", patience=5))

    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
    trainer.test(ckpt_path="best", dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | layer1    | Sequential         | 1.2 K 
1 | layer2    | Sequential         | 4.7 K 
2 | layer3    | Sequential         | 18.6 K
3 | layer4    | Sequential         | 74.1 K
4 | flatten   | Flatten            | 0     
5 | fc        | Sequential         | 558 K 
6 | train_acc | MulticlassAccuracy | 0     
7 | val_acc   | MulticlassAccuracy | 0     
8 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
657 K     Trainable params
0         Non-trainable params
657 K     Total params
2.630     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0 - Train Accuracy: 0.7385


...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 8 - Train Accuracy: 0.9743
Max Train Accuracy Achieved: 0.9753


Restoring states from the checkpoint path at logs/lightning_logs/version_63/checkpoints/epoch=8-step=2286.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/lightning_logs/version_63/checkpoints/epoch=8-step=2286.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       avg_test_acc        │    0.9999999403953552     │
│       avg_test_loss       │   0.002864023670554161    │
│         test_acc          │            1.0            │
│         test_loss         │   0.002882543718442321    │
└───────────────────────────┴───────────────────────────┘

In [ ]:
# Exercise 6: Results
# Train Accuracy: 97.53%
# Test Accuracy: 99.9%